In [11]:
#OSHA DATA
import csv
import copy
import datetime as dt
import matplotlib as plt
import pandas as pd

#Import old osha dataset and convert into a list of lists
csv_data_read = open('old_osha.csv', 'rt')
old_osha = list(csv.reader(csv_data_read))
csv_data_read.close()

#Import new osha dataset and convert into a list of lists
csv_data_read = open('new_osha.csv', 'rt')
new_osha = list(csv.reader(csv_data_read))
csv_data_read.close()

#Create a list of state abbreviations
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA",
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

#Create "osha" to be the combination of both datasets and convert to csv
osha = []

#Dictionary used to build out each row of the osha dataset, values are instantiated as blank strings
row_build = {'year':'', 'month':'', 'day':'','weekday':'', 'state':'', 'city':'', 'zip':'', 'event':''}

#Parse each row of the new osha data and separate field values
#Note: This dataset does not have zip code information
for i in range(1,3648):

    row = copy.copy(row_build)

    #Split the date column into day, month and year and add them to the row dictionary
    split_date = new_osha[i][0].split('/')
    row['month'] = split_date[0]
    row['day'] = split_date[1]
    row['year'] = split_date[2]

    #Calculate day of week
    date = dt.date(int(row['year']), int(row['month']), int(row['day']))
    row['weekday'] = str(dt.datetime.weekday(date))

    #Select city, state, and event description information and add them to the row dictionary
    row['state'] = new_osha[i][2]
    row['city'] = new_osha[i][1]
    row['event'] = new_osha[i][3]

    #append new row to osha dataset using values from the row dictionary
    osha.append([row['year'], row['month'], row['day'], row['weekday'], row['state'], row['city'], row['zip'], row['event']])

#Parse each row of the old osha data and separate field values
#Note: The values of city, state, zip, company are all combined in a single field and must be cleaned
#Note: Separating city name is likely not possible without a database of US cities
for i in range(3,5385):

    row = copy.copy(row_build)

    #Split the date column into day, month and year and add them to the row dictionary
    split_date = old_osha[i][0].split('/')
    row['month'] = split_date[0]
    row['day'] = split_date[1]
    row['year'] = split_date[2]

    #Calculate day of week
    date = dt.date(int(row['year']), int(row['month']), int(row['day']))
    row['weekday'] = str(dt.datetime.weekday(date))

    #Set the event description
    row['event'] = old_osha[i][3]

    #Split and search the generalized location column into constituent parts
    #Typical Column Format: Company Name, City, STATE ZIP
    #Sometimes city names are capitalized, sometimes they aren't
    #There is no guarantee that all of these values are included for each row
    split_loc = old_osha[i][1].split(',')
    split_state_zip = split_loc[len(split_loc)-1].split()

    #Search and remove state and zip code information
    for j in split_state_zip:
        #if the j is in the list of 2-letter state abbreviations
        if j in states:
            row['state'] = j
        #if j only consists of numeric values and has a length of 5
        #This technique may inadvertantly pull street numbers that fit this critera, but the probability is low
        if j.isnumeric() and len(j) == 5:
            row['zip'] = j

    osha.append([row['year'], row['month'], row['day'], row['weekday'], row['state'], row['city'], row['zip'], row['event']])

write = False
if write:
    csv_write = open('osha.csv', 'wt', newline='')
    csv_out = csv.writer(csv_write)
    csv_out.writerows(osha)
    csv_write.close()


weekdaymatch = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
daycount = [0, 0, 0, 0, 0, 0, 0]
for i in osha:
    daycount[int(i[3])] += 1
print(daycount)

plt.pyplot.bar(['M', 'T', 'W', 'Th', 'Fr', 'Sa', 'Su'], daycount)
it = input("End: ")

[1595, 1637, 1603, 1622, 1413, 718, 441]


AttributeError: module 'matplotlib' has no attribute 'pyplot'